In [1]:
## Importing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as linalg
from sklearn import linear_model
from matplotlib import cm
import random

In [2]:
## Functions

## Cholesky equation solver
def Cholesky_solver(M, b):
    c, low = linalg.cho_factor(M)
    return (linalg.cho_solve((c, low), b))

## For solving linear equation
def linear_eqn_solver(m, X, c):
    return (np.dot(X, m) + c)
    
## Main linear regression
def lin_reg(x, y, test):
    
    ## Build matrix X
    x_of1 = np.ones((x.shape[0], 1))
    X = np.hstack((x_of1, x))
    
    ## b <-- X.T y
    b = np.dot(X.T, y)
    
    ## A <-- X.T X
    A = np.dot(X.T, X)
    
    ## Solve AB = b using Cholesky factorization
    B_hat = Cholesky_solver(A, b)
    
    ## Predict
    lin_reg_pred = linear_eqn_solver(B_hat[1:], test, B_hat[0])
    
    return lin_reg_pred, B_hat

In [3]:
## Nabla of J
def nabla_J(w_n, x, y):
    x = np.array(x)
    y = np.array(y)
    x_of1 = np.ones((x.shape[0], 1))
    X = np.hstack((x_of1, x))
    nabla_B = (lin_reg(x, y, w_n))[1]
    ## Vectorizing the eqn
    output = -2*(np.dot(X.T, (y-np.dot(X,nabla_B))))
    return output

## mini batch gradient descent
def mini_batch_gd(x, y, initial_guess, batch_size, learning_rate, tolerance):
    
    ## Making x, y, and zipping into train list
    x = np.asarray(x)
    y = np.asarray(y)
    train = list(zip(x,y))
    
    ## Starting with n = 0
    n = 0
    
    ## w_n is the initial guess passed by the user
    w_n = initial_guess
    
    ## Creating our list of mini_batches 
    mini_batches = [] 
    ## splitting the list of training data into arrays with required batch_size
    mini_batches_temp = np.array_split(train, (len(x)/batch_size)) 
    ## randomizing the array of split training data so we can just iterate through them to get random training batch
    shuffeled_mini_batches = random.sample(mini_batches_temp, len(mini_batches_temp)) 
    
    ## while loop (as per algorithm) + keeping restriction that n must be smaller than (or equal to) the number of split arrays
    while((np.linalg.norm(nabla_J(w_n, x, y)) > tolerance) and (n<len(shuffeled_mini_batches))):
        ## picking first element of random array
        mini_batch = shuffeled_mini_batches[n]
        ## seperating x and y component
        mini_batch_x = [x_val[0] for x_val in mini_batch]
        mini_batch_y = [y_val[1] for y_val in mini_batch]
        ## passing seperated x and y and updating value of w_n
        w_n = w_n + (learning_rate * np.sum(nabla_J(w_n, mini_batch_x, mini_batch_y)))
        ## increasing n
        n = n + 1
        
    return(w_n)

In [6]:
X = [[0], [1], [2], [3], [4]]
x = np.asarray(X)
y = [0, 0.3, 0.75, 1, 2]

initial_guess = 1.9
batch_size = 2
learning_rate = 500
tolerance = 1e-20

print(mini_batch_gd(x, y, initial_guess, batch_size, learning_rate, tolerance))

1.9000000000007908


In [5]:
## Energy Efficiency Data Set from UCI Machine Learning

print("Energy Efficiency Data Set from UCI Machine Learning\n")
df = pd.read_excel('ENB2012_data.xlsx')

## Predict the first three samples of the dataset

X = df.iloc[1:,:-2]
y = df.iloc[1:,-2:]
Actual_y = df.iloc[:1,-2:]
test = df.iloc[:1,:-2]

print(X)

initial_guess = np.array([[12, 20]])
batch_size = 10
learning_rate = 500
tolerance = 1e-20

print("Actual Value\n", Actual_y)
print("\nx-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x\n")

# print(mini_batch_gd(X, y, initial_guess, batch_size, learning_rate, tolerance))

Energy Efficiency Data Set from UCI Machine Learning

       X1     X2     X3      X4   X5  X6   X7  X8
1    0.98  514.5  294.0  110.25  7.0   3  0.0   0
2    0.98  514.5  294.0  110.25  7.0   4  0.0   0
3    0.98  514.5  294.0  110.25  7.0   5  0.0   0
4    0.90  563.5  318.5  122.50  7.0   2  0.0   0
5    0.90  563.5  318.5  122.50  7.0   3  0.0   0
..    ...    ...    ...     ...  ...  ..  ...  ..
763  0.64  784.0  343.0  220.50  3.5   5  0.4   5
764  0.62  808.5  367.5  220.50  3.5   2  0.4   5
765  0.62  808.5  367.5  220.50  3.5   3  0.4   5
766  0.62  808.5  367.5  220.50  3.5   4  0.4   5
767  0.62  808.5  367.5  220.50  3.5   5  0.4   5

[767 rows x 8 columns]
Actual Value
       Y1     Y2
0  15.55  21.33

x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x

